In [1]:
import pandas as pd
import numpy as np
# import plotly.express as px
import plotly.graph_objects as go
from astropy.coordinates import SkyCoord
import astropy.units as u

# For NGC 2682

In [2]:
df_Gaia = pd.read_csv('Gaia_cat.txt')
df_2682 = pd.read_csv('2682.txt')

# Selecting a Cluster from Gaia
df = df_Gaia[df_Gaia['cluster'] == 'NGC_2682'].drop(columns = 'cluster')
df.reset_index(drop = True, inplace = True)
df

,source_id,RAdeg,DEdeg,g_mag,g_rp,qf,P_F6,P_F8,P_F10,P_GMM,class
0,604698306849407232,133.078791,11.397105,14.919369,0.543600,1,0.422395,0.535459,0.893332,NaN,M
1,604698345504195456,133.056635,11.393364,14.428984,0.512750,1,0.302668,0.310425,0.836294,NaN,M
2,604701025563850368,132.975697,11.391630,20.216871,1.138721,1,0.274694,0.309321,0.133937,NaN,F
3,604701128643010816,132.981408,11.402849,13.139835,0.446609,1,0.775353,0.839900,0.941790,NaN,M
4,604701163002747136,133.020764,11.383099,16.562843,0.896111,1,0.741103,0.778229,0.941832,NaN,M
...,...,...,...,...,...,...,...,...,...,...,...
1536,598959822520052480,132.394796,11.788074,14.189812,0.481028,1,0.864449,0.935510,0.982192,NaN,M
1537,598959921303885952,132.414641,11.806690,18.442612,1.102053,1,0.802986,0.922761,0.983147,NaN,M
1538,598959955665887872,132.417570,11.807244,20.633884,1.188070,1,0.467214,0.500625,0.039856,NaN,F
1539,598960642858424576,132.365539,11.815901,19.527473,1.130579,1,0.732178,0.860319,0.966737,NaN,M


In [3]:
# Defining Coordinates
CoordG = SkyCoord(df['RAdeg'], df['DEdeg'], unit = 'deg')
CoordU = SkyCoord(df_2682['RAdeg'], df_2682['DEdeg'], unit = 'deg')

# Matching stars using AstroPy CoordU.search_around_sky function
idxG, idxU, sep, __ = CoordU.search_around_sky(CoordG, seplimit = 1 * u.arcsec)

# Creating New Columns
df['F148'] = 0
df['F154'] = 0
df['F169'] = 0
df['eF148'] = 0
df['eF154'] = 0
df['eF169'] = 0
# df['F148s'] = 0
# df['F154s'] = 0
# df['F169s'] = 0

# Populating new columns using the index
for jdx in np.arange(idxG.shape[0]):
  df.iloc[idxG[jdx], 11] = max(df_2682.iloc[idxU[jdx], 2], df_2682.iloc[idxU[jdx], 3])
  df.iloc[idxG[jdx], 12] = max(df_2682.iloc[idxU[jdx], 5], df_2682.iloc[idxU[jdx], 6])
  df.iloc[idxG[jdx], 13] = max(df_2682.iloc[idxU[jdx], 8], df_2682.iloc[idxU[jdx], 9])
  df.iloc[idxG[jdx], 14] = df_2682.iloc[idxU[jdx], 4]
  df.iloc[idxG[jdx], 15] = df_2682.iloc[idxU[jdx], 7]
  df.iloc[idxG[jdx], 16] = df_2682.iloc[idxU[jdx], 10]
  # df.iloc[idxG[jdx], 17] = df_2682.iloc[idxU[jdx], 3]
  # df.iloc[idxG[jdx], 18] = df_2682.iloc[idxU[jdx], 6]
  # df.iloc[idxG[jdx], 19] = df_2682.iloc[idxU[jdx], 9]
df['F148'] = df['F148'].replace(0, np.nan)
df['F154'] = df['F154'].replace(0, np.nan)
df['F169'] = df['F169'].replace(0, np.nan)
df['eF148'] = df['eF148'].replace(0, np.nan)
df['eF154'] = df['eF154'].replace(0, np.nan)
df['eF169'] = df['eF169'].replace(0, np.nan)
# df

In [4]:
# Constants
co = 0.7 # Cutoff

# create figure
fig = go.Figure()

# Members - [df['P_F10'] > co]
# Candidates - [df['P_F10'] < co][df['P_F6'] > co]
# Field -  [df['P_F10'] < co][df['P_F6'] < co]

# Add surface trace
fig.add_trace(go.Scatter(name = 'Gaia Members (1541)', x=df['g_rp'], y=df['g_mag'], mode='markers', marker = dict(size = 3, color='grey'), hoverinfo = 'skip'))
fig.add_trace(go.Scatter(name = 'F148 Members (84)', x=df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['g_rp'], 
                         y=df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='#D41159', symbol = 101), 
                         text = np.dstack((df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['F148'], df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['eF148'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F148: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F148 Candidates (18)', x=df.loc[(df['F148'] > 0) & (df['P_F10'] < co) & (df['P_F6'] > co)]['g_rp'], 
                         y=df.loc[(df['F148'] > 0) & (df['P_F10'] < co) & (df['P_F6'] > co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='#1A85FF', symbol = 119), 
                         text = np.dstack((df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['F148'], df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['eF148'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F148: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F148 Field (9)', x=df.loc[(df['F148'] > 0) & (df['P_F10'] < co) & (df['P_F6'] < co)]['g_rp'], 
                         y=df.loc[(df['F148'] > 0) & (df['P_F10'] < co) & (df['P_F6'] < co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='black', symbol = 133), 
                         text = np.dstack((df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['F148'], df.loc[(df['F148'] > 0) & (df['P_F10'] > co)]['eF148'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F148: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F154 Members (31)', x=df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['g_rp'], 
                         y=df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='#D41159', symbol = 101), visible = False, 
                         text = np.dstack((df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['F154'], df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['eF154'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F154: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F154 Candidates (7)', x=df.loc[(df['F154'] > 0) & (df['P_F10'] < co) & (df['P_F6'] > co)]['g_rp'], 
                         y=df.loc[(df['F154'] > 0) & (df['P_F10'] < co) & (df['P_F6'] > co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='#1A85FF', symbol = 119), visible = False, 
                         text = np.dstack((df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['F154'], df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['eF154'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F154: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F154 Field (6)', x=df.loc[(df['F154'] > 0) & (df['P_F10'] < co) & (df['P_F6'] < co)]['g_rp'], 
                         y=df.loc[(df['F154'] > 0) & (df['P_F10'] < co) & (df['P_F6'] < co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='black', symbol = 133), visible = False, 
                         text = np.dstack((df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['F154'], df.loc[(df['F154'] > 0) & (df['P_F10'] > co)]['eF154'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F154: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F169 Members (58)', x=df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['g_rp'], 
                         y=df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['g_mag'], mode='markers', marker = dict(size = 8, color='#D41159', symbol = 101), visible = False, 
                         text = np.dstack((df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['F169'], df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['eF169'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F169: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F169 Candidates (15)', x=df.loc[(df['F169'] > 0) & (df['P_F10'] < co) & (df['P_F6'] > co)]['g_rp'], 
                         y=df.loc[(df['F169'] > 0) & (df['P_F10'] < co) & (df['P_F6'] > co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='#1A85FF', symbol = 119), visible = False, 
                         text = np.dstack((df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['F169'], df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['eF169'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F169: %{text[1]:.2f}±%{text[2]:.2f} mag"))
fig.add_trace(go.Scatter(name = 'F169 Field (6)', x=df.loc[(df['F169'] > 0) & (df['P_F10'] < co) & (df['P_F6'] < co)]['g_rp'], 
                         y=df.loc[(df['F169'] > 0) & (df['P_F10'] < co) & (df['P_F6'] < co)]['g_mag'], 
                         mode='markers', marker = dict(size = 8, color='black', symbol = 133), visible = False, 
                         text = np.dstack((df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['source_id'], df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['F169'], df.loc[(df['F169'] > 0) & (df['P_F10'] > co)]['eF169'])).squeeze(),
                         hovertemplate="<b>Source ID: <br>%{text[0]:}</b><br>G: %{y:.2f} mag<br>G - G_RP: %{x:.2f} mag<br>F169: %{text[1]:.2f}±%{text[2]:.2f} mag"))

# Update plot sizing
fig.update_layout(
    width = 600,
    height = 800,
    # autosize=False,
    margin=dict(t=30, b=0, l=0, r=0),
)

fig.update_layout(
        title={
        'text': 'Gaia CMD of NGC 2682 with UVIT sources',
        'y':0.99,
        'x':0.44,
        # 'xanchor': 'center',
        # 'yanchor': 'top'
        })
fig.update_xaxes(
        title_text = "G - G<sub>RP</sub> [mag]",
        range=[-0.3, 1.5]
        )
fig.update_yaxes(
        title_text = "G [mag]",
        autorange="reversed"
        )


# Add dropdown
fig.update_layout(
    updatemenus = list([
        dict(active=0,
            showactive = True,
            buttons=list([   
                dict(label = "F148",
                    method = "update",
                    args = [{"visible": [True, True, True, True, False, False, False, False, False, False]}]), # show F148
                dict(label = "F154",
                    method = "update",
                    args = [{"visible": [True, False, False, False, True, True, True, False, False, False]}]), # show F154
                dict(label = "F169",
                    method = "update",
                    args = [{"visible": [True, False, False, False, False, False, False, True, True, True]}]), # show F169
                ]),
            direction="down",
            # pad={"r": 10, "t": 10},
            x=0.86,
            xanchor="left",
            y=0.96,
            yanchor="top")])
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="UVIT Filter:", showarrow=False,
        x=1.36, y=0.99, yref="paper", align="left")
    ],
    showlegend = True,
    legend=dict(
    yanchor="top",
    y=0.98,
    xanchor="left",
    x=0.02
))

config = dict({
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['toggleSpikelines','hoverCompareCartesian', 'autoScale2d', 'select2d', 'lasso2d', 'hoverClosestCartesian', 'zoomIn2d', 'zoomOut2d']
})

fig.show(config = config)

In [5]:
fig.write_html("NGC2682_Final.html")